In [31]:
# select * from table where text like '%internet%'

# What’s the internet usage policy?

# Source of information: All company policies documents.
# Relevant Documents for user query - This will be passed as a Context to the LLM model. (And then LLM will generate a response based on this context)

# Step 1: Define Sample Documents
# Document Corpus
# documents = [
    
#     {"doc_id": "1", "section": "Pay Policies", "content": "Employees are paid bi-weekly via direct deposit."},
#     {"section": "Leave of Absence", "content": "Employees must submit a leave request for approval."},
#     # query = "What’s the internet usage policy?"
#     {"section": "Internet Use", "content": "Company internet must be used for work-related tasks only."},
#     {"section": "Internet Use", "content": "Company internet is a broadband internet."},
#     {"section": "Break at Work", "content": "Employees can take an hour break."},
#     {"section": "Harassment", "content": "Interact with each employee with Respect"}
# ]

documents = [
        "Employees are paid bi-weekly via direct deposit.",
        "Employees must submit a leave request for approval.", 
        "Company internet must be used for work-related tasks only.",
        "Company internet is a broadband internet.",
        "Employees can take an hour break.",
        "Interact with each employee with Respect"
]
# # Step 2: Get Content Texts
# content_corpus = [doc for doc in documents]

# content_corpus
content_corpus = documents
content_corpus

['Employees are paid bi-weekly via direct deposit.',
 'Employees must submit a leave request for approval.',
 'Company internet must be used for work-related tasks only.',
 'Company internet is a broadband internet.',
 'Employees can take an hour break.',
 'Interact with each employee with Respect']

In [ ]:
# Goal: Retrieve relevant documents based on user query (a.k.a. Context Retrieval)

# %pip install -q sentence-transformers

In [32]:
from sentence_transformers import SentenceTransformer

model = SentenceTransformer("all-MiniLM-L6-v2")
doc_vectors = model.encode(content_corpus)

In [35]:
doc_vectors

array([[ 0.02472517, -0.00908147,  0.0388713 , ...,  0.01965641,
         0.04260008, -0.02707142],
       [ 0.03315504,  0.04853383,  0.04736274, ...,  0.10182008,
         0.09159282,  0.00358369],
       [-0.07135904, -0.03066471,  0.03183771, ..., -0.04109798,
         0.06524782, -0.00688535],
       [-0.00383736, -0.02336751,  0.02958677, ..., -0.04415291,
         0.12559088, -0.03139851],
       [-0.01790445,  0.0149585 ,  0.08163827, ..., -0.03217234,
        -0.00513649,  0.05279536],
       [-0.00240888,  0.03361145, -0.06162645, ...,  0.04830879,
         0.03707644, -0.01683046]], shape=(6, 384), dtype=float32)

In [21]:
# Step 3: User Query and Semantic Matching

query = "What’s the internet usage policy?"
query_vec = model.encode([query])[0]
query_vec

array([ 2.39818310e-03, -4.11840044e-02, -2.52490286e-02, -4.67050411e-02,
        4.32103127e-03,  1.65876560e-02,  1.20891713e-01, -3.50352786e-02,
        2.16827448e-03, -1.62921249e-04,  2.62875594e-02,  9.05027986e-02,
       -2.66066380e-02, -1.82131920e-02,  3.06277741e-02,  1.67854931e-02,
        1.55614391e-02, -8.26497525e-02, -3.40456367e-02, -3.08671594e-02,
        7.89995566e-02, -3.16904113e-02,  1.35831609e-02,  9.12292919e-04,
       -1.05809094e-02,  3.91191058e-02, -3.48707438e-02,  8.64372851e-05,
       -3.52702960e-02,  3.56902406e-02,  9.55272093e-03, -3.57899368e-02,
        4.84950189e-03, -4.10227031e-02, -7.66861811e-02, -1.00646727e-01,
       -9.23561379e-02, -2.47274316e-03, -2.74321493e-02,  2.85045654e-02,
        2.86296215e-02, -7.78359026e-02, -2.46462598e-03,  9.98250619e-02,
        5.86107075e-02,  2.24836338e-02,  1.56646027e-04,  1.44218104e-02,
        5.55959996e-04,  3.22429687e-02,  1.03300057e-01,  3.42919938e-02,
        3.94683331e-02,  

In [22]:
similarities = model.similarity(query_vec, doc_vectors)

# Ensure it's a 1D numpy array
import numpy as np
similarities = np.asarray(similarities).squeeze()
similarities

array([0.12655608, 0.09747534, 0.447553  , 0.47982502, 0.11316214,
       0.04429522], dtype=float32)

In [23]:

# Now get top 3
top_3_indices = np.argsort(similarities)[::-1][:3]
print(top_3_indices)
top_scores = similarities[top_3_indices]
top_scores

[3 2 0]


array([0.47982502, 0.447553  , 0.12655608], dtype=float32)

In [24]:
top_scores

array([0.47982502, 0.447553  , 0.12655608], dtype=float32)

In [ ]:
top_docs = [documents[i] for i in top_3_indices]
# documents = [
#     {"section": "Pay Policies", "content": "Employees are paid bi-weekly via direct deposit."},
#     {"section": "Leave of Absence", "content": "Employees must submit a leave request for approval."},
#     {"section": "Internet Use", "content": "Company internet must be used for work-related tasks only."},
#     {"section": "Break at Work", "content": "Employees can take an hour break."},
#     {"section": "Harassment", "content": "Interact with each employee with Respect"}
# ]

print (top_docs)
context = ", ".join(top_docs)
context

['Company internet is a broadband internet.', 'Company internet must be used for work-related tasks only.', 'Employees are paid bi-weekly via direct deposit.']


'Company internet is a broadband internet., Company internet must be used for work-related tasks only., Employees are paid bi-weekly via direct deposit.'

In [26]:
import os
from dotenv import load_dotenv
from openai import OpenAI

load_dotenv(override=True, dotenv_path="../.env.local")
my_api_key = os.getenv("OPENAI_API_KEY")

my_client = OpenAI(api_key=my_api_key)
# my_client

def ask_question_open_ai(prompt):

    # print(f"User asked: {prompt}")
    # my_client.chat.completions.create

    llm_response = my_client.chat.completions.create(
        model="gpt-5-nano",
        # messages=[
        #     {"role": "system", "content": "You are a helpful assistant. Answer as concisely as possible."},
        #     {"role": "user", "content": prompt}
        # ]
        messages=[
            {"role": "system", "content": '''
             You are an assistant who answers only based on the given context.
             '''},
            {"role": "user", "content": f"Context: {context}\n\n User Question: {query}"} 
        ]

    )
    return llm_response.choices[0].message.content  


In [27]:
print (query)
response = ask_question_open_ai(query)

What’s the internet usage policy?


In [28]:
print(f"User query: {query}")
print(f"Context: {context}")

print(f"\n\nOpen AI Response: {response}")

User query: What’s the internet usage policy?
Context: Company internet is a broadband internet., Company internet must be used for work-related tasks only., Employees are paid bi-weekly via direct deposit.


Open AI Response: - The company internet is broadband.
- It must be used for work-related tasks only (non-work use is not allowed).


In [ ]:
#Reference Data
expected_answer = "The company's internet must be used for work-related tasks only."    
actual_response = response

# Call ChatCompltions API to compare expected vs actual
#LLM AS A JUDGE
comparison_prompt = f"""
You are an expert evaluator. Compare the actual response to the expected answer and determine if they match in meaning.
Expected Answer: {expected_answer}
Actual Response: {actual_response}
Do they match in meaning? Answer with 'Yes' or 'No' and provide a brief explanation.
"""
comparison_response = my_client.chat.completions.create(
    model="gpt-5-nano",     
    messages=[
        {"role": "system", "content": "You are an expert evaluator."},
        {"role": "user", "content": comparison_prompt}
    ]
)   
print(f"\nExpected Answer: {comparison_response.choices[0].message.content}")

In [ ]:
print (f"\nExpected Answer: {expected_answer}")
print (f"\nActual Response: {actual_response}")